# Transcript from Lecture, October 14, 2021


In [1]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2021-fall/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
%matplotlib ipympl

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d




np.set_printoptions(precision = 4)

# Condition number, residual, and error

In [2]:
A = np.random.random((10,10))

In [3]:
x_exact = np.ones(A.shape[1])
b = A @ x_exact
print('b:', b)

b: [6.2759 5.5063 5.0857 4.3974 4.0739 4.9684 3.994  7.7254 6.69   5.2455]


In [4]:
x = npla.solve(A, b)

print('x:', x)


x: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [5]:
residual = b - A @ x
error = x_exact - x

print('n:', A.shape[1])
print()
print('relative residual norm:', npla.norm(residual)/npla.norm(b))
print('relative error norm:', npla.norm(error)/npla.norm(x_exact))
print()
print('condition number of A:', npla.cond(A, 2))

n: 10

relative residual norm: 1.549061503249027e-16
relative error norm: 5.897886751604628e-15

condition number of A: 399.9774785558365


# The Hilbert matrix, an ill-conditioned example

In [6]:
def hilbert(n):
    """n-by-n Hilbert matrix, a famous example of an ill-conditioned matrix"""
    A = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            A[i,j] = 1 / (i + j + 1)
    return A

In [7]:
print('hilbert(4):\n', hilbert(4))

hilbert(4):
 [[1.     0.5    0.3333 0.25  ]
 [0.5    0.3333 0.25   0.2   ]
 [0.3333 0.25   0.2    0.1667]
 [0.25   0.2    0.1667 0.1429]]


In [8]:
U, S, Vt = npla.svd(hilbert(4))
print('singular values:', S)

singular values: [1.5002e+00 1.6914e-01 6.7383e-03 9.6702e-05]


In [9]:
A = hilbert(4)
npla.matrix_rank(A)

4

In [10]:
npla.norm(A,2)

1.5002142800592426

In [11]:
npla.cond(A,2)

15513.738738931106

In [13]:
A = hilbert(10)
print('hilbert(10):\n', hilbert(10))
print()
U, S, Vt = npla.svd(A)
print('singular values:', S)

hilbert(10):
 [[1.     0.5    0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1   ]
 [0.5    0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909]
 [0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909 0.0833]
 [0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769]
 [0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769 0.0714]
 [0.1667 0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769 0.0714 0.0667]
 [0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769 0.0714 0.0667 0.0625]
 [0.125  0.1111 0.1    0.0909 0.0833 0.0769 0.0714 0.0667 0.0625 0.0588]
 [0.1111 0.1    0.0909 0.0833 0.0769 0.0714 0.0667 0.0625 0.0588 0.0556]
 [0.1    0.0909 0.0833 0.0769 0.0714 0.0667 0.0625 0.0588 0.0556 0.0526]]

singular values: [1.7519e+00 3.4293e-01 3.5742e-02 2.5309e-03 1.2875e-04 4.7297e-06
 1.2290e-07 2.1474e-09 2.2667e-11 1.0932e-13]


In [14]:
x_exact = np.ones(A.shape[1])
b = A @ x_exact
b

array([2.929 , 2.0199, 1.6032, 1.3468, 1.1682, 1.0349, 0.9307, 0.8467,
       0.7773, 0.7188])

In [15]:
x = npla.solve(A, b)

print('x:', x)

x: [1.     1.     1.     1.     1.0002 0.9995 1.0008 0.9992 1.0004 0.9999]


In [16]:
residual = b - A @ x
error = x_exact - x

print('n:', A.shape[1])
print()
print('relative residual norm:', npla.norm(residual)/npla.norm(b))
print('relative error norm:', npla.norm(error)/npla.norm(x_exact))
print()
print('condition number of A:', npla.cond(A, 2))

n: 10

relative residual norm: 5.2709532231075846e-17
relative error norm: 0.0004170709495986796

condition number of A: 16024253208221.582


In [17]:
A = hilbert(20)
x_exact = np.ones(A.shape[1])
b = A @ x_exact
b

array([3.5977, 2.6454, 2.1908, 1.901 , 1.6926, 1.5326, 1.4044, 1.2986,
       1.2093, 1.1327, 1.066 , 1.0074, 0.9553, 0.9087, 0.8666, 0.8286,
       0.7938, 0.762 , 0.7328, 0.7058])

In [18]:
x = npla.solve(A, b)

print('x:', x)

x: [  1.       1.       0.9984   1.0279   0.7395   2.4999  -4.7311  16.2389
 -28.0164  40.8394 -38.1484  31.7326 -33.3383  55.5837 -52.7371   4.7578
  52.4796 -55.3243  27.1172  -3.7195]


In [19]:
residual = b - A @ x
error = x_exact - x

print('n:', A.shape[1])
print()
print('relative residual norm:', npla.norm(residual)/npla.norm(b))
print('relative error norm:', npla.norm(error)/npla.norm(x_exact))
print()
print('condition number of A:', npla.cond(A, 2))

n: 20

relative residual norm: 5.116979263680122e-16
relative error norm: 30.627485857988308

condition number of A: 2.6937874400784947e+18


In [20]:
U, S, Vt = npla.svd(A)
print('singular values:', S)

singular values: [1.9071e+00 4.8704e-01 7.5596e-02 8.9611e-03 8.6767e-04 7.0334e-05
 4.8305e-06 2.8277e-07 1.4140e-08 6.0361e-10 2.1929e-11 6.7408e-13
 1.7383e-14 3.7358e-16 1.4048e-17 1.1844e-17 7.0744e-18 3.8403e-18
 2.5573e-18 9.0534e-19]


In [21]:
npla.matrix_rank(A)

13

# Jacobi iterative method for Ax = b

In [22]:
A = np.array([[3,1],[-2,4]])
A

array([[ 3,  1],
       [-2,  4]])

In [23]:
b = np.array([9,8])
b

array([9, 8])

In [24]:
npla.solve(A,b)

array([2., 3.])

In [25]:
cs111.LUsolve(A,b)

(array([2., 3.]), 0.0)

In [26]:
# Start the iteration with x = 0 as our first guess
x = np.zeros(2)
x

array([0., 0.])

In [27]:
# How good is x? Compute the relative residual norm
relres = npla.norm(b - A@x) / npla.norm(b)
relres

1.0

In [28]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [3. 2.]
relres: 0.5252257314388903


In [29]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [2.3333 3.5   ]
relres: 0.1666666666666666


In [30]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [1.8333 3.1667]
relres: 0.08753762190648182


In [31]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [1.9444 2.9167]
relres: 0.027777777777777832


In [32]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [2.0278 2.9722]
relres: 0.014589603651080349


In [33]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [2.0093 3.0139]
relres: 0.00462962962962965


In [34]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [1.9954 3.0046]
relres: 0.00243160060851345


In [ ]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

In [35]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [1.9985 2.9977]
relres: 0.0007716049382715942


In [36]:
# Try to improve the guess
x = np.array([(9-x[1])/3, (8+2*x[0])/4])
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [2.0008 2.9992]
relres: 0.0004052667680856294


In [37]:
npla.cond(A)

1.4560832005096076

# Matrix view of Jacobi

In [38]:
A

array([[ 3,  1],
       [-2,  4]])

In [39]:
d = A.diagonal()
d

array([3, 4])

In [40]:
D = np.diag(d)
D

array([[3, 0],
       [0, 4]])

In [41]:
C = A - D
C

array([[ 0,  1],
       [-2,  0]])

In [42]:
# initial guess x = 0
x = np.zeros(2)
x

array([0., 0.])

In [43]:
# try to improve the guess, matrix version
x = (b - C@x) / d
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [3. 2.]
relres: 0.5252257314388903


In [44]:
# try to improve the guess, matrix version
x = (b - C@x) / d
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [2.3333 3.5   ]
relres: 0.1666666666666666


In [45]:
# try to improve the guess, matrix version
x = (b - C@x) / d
print('x:',x)

relres = npla.norm(A@x-b)/npla.norm(b)
print('relres:',relres)

x: [1.8333 3.1667]
relres: 0.08753762190648182


In [46]:
x = np.zeros(2)
for i in range(25):
    x = (b - C@x) / d
    relres = npla.norm(A@x-b)/npla.norm(b)
    print('iteration', i, 'x:', x, ', relres:' ,relres)

iteration 0 x: [3. 2.] , relres: 0.5252257314388903
iteration 1 x: [2.3333 3.5   ] , relres: 0.1666666666666666
iteration 2 x: [1.8333 3.1667] , relres: 0.08753762190648182
iteration 3 x: [1.9444 2.9167] , relres: 0.027777777777777832
iteration 4 x: [2.0278 2.9722] , relres: 0.014589603651080349
iteration 5 x: [2.0093 3.0139] , relres: 0.00462962962962965
iteration 6 x: [1.9954 3.0046] , relres: 0.00243160060851345
iteration 7 x: [1.9985 2.9977] , relres: 0.0007716049382715942
iteration 8 x: [2.0008 2.9992] , relres: 0.0004052667680856294
iteration 9 x: [2.0003 3.0004] , relres: 0.0001286008230453045
iteration 10 x: [1.9999 3.0001] , relres: 6.754446134775651e-05
iteration 11 x: [2.     2.9999] , relres: 2.143347050755075e-05
iteration 12 x: [2. 3.] , relres: 1.1257410224711609e-05
iteration 13 x: [2. 3.] , relres: 3.572245084626502e-06
iteration 14 x: [2. 3.] , relres: 1.8762350374519347e-06
iteration 15 x: [2. 3.] , relres: 5.953741808037523e-07
iteration 16 x: [2. 3.] , relres: 3.12

In [47]:
x, rr = cs111.Jsolve(A,b)

In [48]:
x

array([2., 3.])

In [49]:
rr

[1.0,
 0.5252257314388903,
 0.1666666666666666,
 0.08753762190648182,
 0.027777777777777832,
 0.014589603651080349,
 0.00462962962962965,
 0.00243160060851345,
 0.0007716049382715942,
 0.0004052667680856294,
 0.0001286008230453045,
 6.754446134775651e-05,
 2.143347050755075e-05,
 1.1257410224711609e-05,
 3.572245084626502e-06,
 1.8762350374519347e-06,
 5.953741808037523e-07,
 3.1270583952089815e-07,
 9.922902990936187e-08,
 5.211764003288578e-08,
 1.653817172098115e-08,
 8.686273338814296e-09]

In [50]:
cs111.Jsolve?